In [16]:
import pandas as pd
import numpy as np
from nltk import bigrams
import nltk
from nltk.tokenize import word_tokenize
import nltk
from nltk.tokenize import sent_tokenize
import re
from nltk import trigrams
from textstat import flesch_reading_ease
nltk.download('punkt')

# Load the two-word prevalence table
two_word_prevalence_table = pd.read_csv('two_word_prevalence_table.csv')
two_word_prevalence_dict = two_word_prevalence_table.set_index('Bigram')['Prevalence_Factor'].to_dict()

prevalence_table = pd.read_csv('prevalence_table.csv')
prevalence_dict = prevalence_table.set_index('Word')['Prevalence_Factor'].to_dict()

# Load the three-word prevalence table
three_word_prevalence_table = pd.read_csv('three_word_prevalence_table.csv')
three_word_prevalence_dict = three_word_prevalence_table.set_index('Trigram')['Prevalence_Factor'].to_dict()

four_word_prevalence_table = pd.read_csv('four_word_prevalence_table.csv')
four_word_prevalence_dict = four_word_prevalence_table.set_index('Fourgram')['Prevalence_Factor'].to_dict()


def get_sentence_prevalence_array(text):
    sentences = sent_tokenize(text)
    clean_sentences = [clean_text(sentence) for sentence in sentences]
    output = []
    for sentence in clean_sentences:
        words = word_tokenize(sentence)
        prevalences = [get_prevalence_word(word) for word in words]
        prevalence_array = prevalences[:30] + [0] * (30 - len(prevalences))
        output.append(prevalence_array)
    return output

def get_sentence_prevalence_array(text):
    sentences = sent_tokenize(text)
    clean_sentences = [clean_text(sentence) for sentence in sentences if len(word_tokenize(sentence)) <= 30]  # Only consider sentences with 30 or fewer words
    output = []
    for sentence in clean_sentences:
        words = word_tokenize(sentence)
        prevalences = [get_prevalence_word(word) for word in words]
        prevalence_array = prevalences[:30] + [0] * (30 - len(prevalences))
        output.append(prevalence_array)
    return output

def get_prevalence_array(text):
    text = clean_text(text)  # Clean the text using the previously defined function
    words = word_tokenize(text)  # Tokenize the text into words
    prevalences = [get_prevalence_word(word) for word in words]  # Get prevalence scores for each word
    # Ensure the list has 1000 cells, filling empty cells with 0
    prevalence_array = prevalences[:1000] + [0] * (1000 - len(prevalences))
    return prevalence_array

def clean_text(text):
    """Clean the text for analysis."""
    text = text.lower()  # Convert text to lowercase
    text = re.sub(r'[^a-z0-9\s]', '', text)  # Remove punctuation
    return text

def get_prevalence_word(word):
    word = str(word)
    word = word.lower()
    if word in prevalence_dict:
        return prevalence_dict[word]
    else:
        return 0

def get_text_length(text):
    text = str(text)
    return len(text.split())




[nltk_data] Downloading package punkt to /Users/agus/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

# Assuming the get_sentence_prevalence_array function is defined as discussed earlier

def transform_dataset(input_path, output_path):
    # Load dataset
    data = pd.read_csv(input_path)
    
    transformed_data = []
    for index, row in data.iterrows():
        text = row['text']
        generated = row['generated']
        sentence_prevalences = get_sentence_prevalence_array(text)
        
        for sentence_array in sentence_prevalences:
            # Create a dictionary for each sentence with its prevalence array and generated status
            sentence_data = {'generated': generated}
            for i, prevalence in enumerate(sentence_array, start=1):
                sentence_data[f'word_{i}'] = prevalence
            transformed_data.append(sentence_data)
    
    # Convert list of dictionaries to DataFrame
    transformed_df = pd.DataFrame(transformed_data)
    
    # Save the transformed DataFrame to a new CSV file
    transformed_df.to_csv(output_path, index=False)

# Define your input and output file paths
input_path = 'final_test2.csv'
output_path = 'final_test2_sentences.csv'

# Call the function to transform the dataset
transform_dataset(input_path, output_path)


[nltk_data] Downloading package punkt to /Users/agus/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:

import pandas as pd


# Load the dataset
dataset = pd.read_csv('final_test2.csv')
datset = dataset[dataset['text'].apply(lambda x: get_text_length(x) > 0 and get_text_length(x) <= 1000)]

# Apply get_prevalence_array to each text in the dataset
prevalence_arrays = dataset['text'].apply(get_prevalence_array)

# Convert list of lists into a DataFrame
prevalence_df = pd.DataFrame(prevalence_arrays.tolist(), columns=[str(i) for i in range(1, 1001)])

# Prefix the 'generated' column from the original dataset
new_data = pd.concat([dataset['generated'].reset_index(drop=True), prevalence_df], axis=1)

# Save the new dataset to a CSV file
new_data.to_csv('transformed_final_test2.csv', index=False)


In [22]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Load the training dataset
path_train = "transformed_megaset4_sentences.csv"
train_data = pd.read_csv(path_train)

# train_data_dropped = train_data.drop(train_data[train_data['generated'] == 1].sample(n=1).index)
# train_data = train_data[train_data['text'].apply(lambda x: get_text_length(x) >= 300 and get_text_length(x) <= 1000)]

path_test = "final_test2_sentences.csv"
test_data = pd.read_csv(path_test).sample(100000)
# test_data = test_data[test_data['text'].apply(lambda x: get_text_length(x) >= 300 and get_text_length(x) <= 10000)]


# Initialize the StandardScaler
scaler = StandardScaler()

def tostr(x):
    return "word_"+str(x)

x_features = [tostr(i) for i in range(1, 30)]

# Prepare the features and target for the training data
X_train = train_data[x_features]
X_train_scaled = scaler.fit_transform(X_train)
y_train = train_data['generated']

# Prepare the features and target for the testing data
X_test = test_data[x_features]
X_test_scaled = scaler.transform(X_test)
y_test = test_data['generated']

# Initialize MLPClassifier with a specified architecture

mlp_clf = MLPClassifier(hidden_layer_sizes=(100,100,100,100,100,100,100,100), activation='relu', 
                        solver='adam', max_iter=200, random_state=12, verbose=True)

# Train the classifier on the scaled training data
mlp_clf.fit(X_train_scaled, y_train)

# Make predictions on the scaled testing data
y_pred_test = mlp_clf.predict(X_test_scaled)
# Make predictions on the scaled training data for overfitting check
y_pred_train = mlp_clf.predict(X_train_scaled)

# Evaluate the accuracy on the testing data
accuracy_test = accuracy_score(y_test, y_pred_test)
# Evaluate the accuracy on the training data
accuracy_train = accuracy_score(y_train, y_pred_train)

# Output the accuracies
print(f"Accuracy on the training set: {accuracy_train * 100:.2f}%")
print(f"Accuracy on the test set: {accuracy_test * 100:.2f}%")

# If the accuracy on the training set is significantly higher than on the test set,
# it may indicate overfitting.

# Calculate the confusion matrix for test data to get false positive and false negative rates
conf_matrix_test = confusion_matrix(y_test, y_pred_test)
tn, fp, fn, tp = conf_matrix_test.ravel()

# Calculate rates
false_positive_rate = fp / (fp + tn) if (fp + tn) > 0 else 0
false_negative_rate = fn / (fn + tp) if (fn + tp) > 0 else 0

# Print the rates
print(f"False Positive Rate on test data: {false_positive_rate * 100:.2f}%")
print(f"False Negative Rate on test data: {false_negative_rate * 100:.2f}%")
print(f"Average error rate on test data: {(false_positive_rate*100 + false_negative_rate*100) / 2 :.2f}%")

score = 1 / false_negative_rate + 1 / false_positive_rate
print(f"Score: {score:.2f}")

# Print the confusion matrix for the test data
print("Confusion Matrix for the test data:")
print(conf_matrix_test)

Iteration 1, loss = 0.34968978
Iteration 2, loss = 0.34296998
Iteration 3, loss = 0.33935855
Iteration 4, loss = 0.33729877
Iteration 5, loss = 0.33557947
Iteration 6, loss = 0.33391905
Iteration 7, loss = 0.33250285
Iteration 8, loss = 0.33130897
Iteration 9, loss = 0.33036492
Iteration 10, loss = 0.32940762
Iteration 11, loss = 0.32859745
Iteration 12, loss = 0.32778653
Iteration 13, loss = 0.32706837
Iteration 14, loss = 0.32640174
Iteration 15, loss = 0.32580664
Iteration 16, loss = 0.32519086
Iteration 17, loss = 0.32460011
Iteration 18, loss = 0.32411817
Iteration 19, loss = 0.32366582
Iteration 20, loss = 0.32307092
Iteration 21, loss = 0.32265984
Iteration 22, loss = 0.32219341
Iteration 23, loss = 0.32170646
Iteration 24, loss = 0.32143901
Iteration 25, loss = 0.32095724
Iteration 26, loss = 0.32064097
Iteration 27, loss = 0.32039053
Iteration 28, loss = 0.31998099
Iteration 29, loss = 0.31976554
Iteration 30, loss = 0.31954190
Iteration 31, loss = 0.31927056
Iteration 32, los

/opt/homebrew/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Accuracy on the training set: 86.95%
Accuracy on the test set: 83.90%
False Positive Rate on test data: 8.97%
False Negative Rate on test data: 31.45%
Average error rate on test data: 20.21%
Score: 14.32
Confusion Matrix for the test data:
[[62175  6130]
 [ 9969 21726]]


In [27]:
import pandas as pd
from sklearn.metrics import confusion_matrix

# Assuming the train_data is loaded as shown previously
path_train = "final_test2_sentences.csv"
train_data = pd.read_csv(path_train)

# Define the predict function
def predict(sentence):
    prevalence = sum(sentence)
    return 1 if prevalence > 0 else 0

# Extract sentence prevalence factors and the 'generated' column
sentence_columns = [f'word_{i}' for i in range(1, 31)]  # Adjust based on actual column names
X = train_data[sentence_columns].values
y_true = train_data['generated'].values

# Make predictions
y_pred = [predict(row) for row in X]

# Calculate accuracy
correct_predictions = sum(int(pred == true) for pred, true in zip(y_pred, y_true))
accuracy = correct_predictions / len(y_true)

# Calculate confusion matrix to get false positive and false negative rates
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

# Calculate rates
false_positive_rate = fp / (fp + tn)
false_negative_rate = fn / (fn + tp)

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"False Positive Rate: {false_positive_rate * 100:.2f}%")
print(f"False Negative Rate: {false_negative_rate * 100:.2f}%")


Accuracy: 80.13%
False Positive Rate: 15.40%
False Negative Rate: 29.44%


In [6]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Load the training dataset
path_train = "transformed_megaset4_sentences.csv"
train_data = pd.read_csv(path_train)

# train_data_dropped = train_data.drop(train_data[train_data['generated'] == 1].sample(n=1).index)
# train_data = train_data[train_data['text'].apply(lambda x: get_text_length(x) >= 300 and get_text_length(x) <= 1000)]

path_test = "final_test2_sentences.csv"
test_data = pd.read_csv(path_test).sample(100000)
# test_data = test_data[test_data['text'].apply(lambda x: get_text_length(x) >= 300 and get_text_length(x) <= 10000)]


# Initialize the StandardScaler
scaler = StandardScaler()

def tostr(x):
    return "word_"+str(x)

x_features = [tostr(i) for i in range(1, 30)]

# Prepare the features and target for the training data
X_train = train_data[x_features]
X_train_scaled = scaler.fit_transform(X_train)
y_train = train_data['generated']

# Prepare the features and target for the testing data
X_test = test_data[x_features]
X_test_scaled = scaler.transform(X_test)
y_test = test_data['generated']

# Initialize MLPClassifier with a specified architecture

mlp_clf2 = MLPClassifier(hidden_layer_sizes=(2), activation='relu', 
                        solver='adam', max_iter=999999, random_state=12, verbose=True)

# Train the classifier on the scaled training data
mlp_clf2.fit(X_train_scaled, y_train)

# Make predictions on the scaled testing data
y_pred_test = mlp_clf2.predict(X_test_scaled)
# Make predictions on the scaled training data for overfitting check
y_pred_train = mlp_clf2.predict(X_train_scaled)

# Evaluate the accuracy on the testing data
accuracy_test = accuracy_score(y_test, y_pred_test)
# Evaluate the accuracy on the training data
accuracy_train = accuracy_score(y_train, y_pred_train)

# Output the accuracies
print(f"Accuracy on the training set: {accuracy_train * 100:.2f}%")
print(f"Accuracy on the test set: {accuracy_test * 100:.2f}%")

# If the accuracy on the training set is significantly higher than on the test set,
# it may indicate overfitting.

# Calculate the confusion matrix for test data to get false positive and false negative rates
conf_matrix_test = confusion_matrix(y_test, y_pred_test)
tn, fp, fn, tp = conf_matrix_test.ravel()

# Calculate rates
false_positive_rate = fp / (fp + tn) if (fp + tn) > 0 else 0
false_negative_rate = fn / (fn + tp) if (fn + tp) > 0 else 0

# Print the rates
print(f"False Positive Rate on test data: {false_positive_rate * 100:.2f}%")
print(f"False Negative Rate on test data: {false_negative_rate * 100:.2f}%")
print(f"Average error rate on test data: {(false_positive_rate*100 + false_negative_rate*100) / 2 :.2f}%")

score = 1 / false_negative_rate + 1 / false_positive_rate
print(f"Score: {score:.2f}")

# Print the confusion matrix for the test data
print("Confusion Matrix for the test data:")
print(conf_matrix_test)


Iteration 1, loss = 0.38253036
Iteration 2, loss = 0.35646761
Iteration 3, loss = 0.35583621
Iteration 4, loss = 0.35540517
Iteration 5, loss = 0.35519821
Iteration 6, loss = 0.35486349
Iteration 7, loss = 0.35483404
Iteration 8, loss = 0.35472635
Iteration 9, loss = 0.35466861
Iteration 10, loss = 0.35467789
Iteration 11, loss = 0.35467291
Iteration 12, loss = 0.35467595
Iteration 13, loss = 0.35465823
Iteration 14, loss = 0.35465838
Iteration 15, loss = 0.35464592
Iteration 16, loss = 0.35461727
Iteration 17, loss = 0.35397195
Iteration 18, loss = 0.35392455
Iteration 19, loss = 0.35391706
Iteration 20, loss = 0.35391713
Iteration 21, loss = 0.35389435
Iteration 22, loss = 0.35390159
Iteration 23, loss = 0.35390849
Iteration 24, loss = 0.35388773
Iteration 25, loss = 0.35389409
Iteration 26, loss = 0.35388803
Iteration 27, loss = 0.35389251
Iteration 28, loss = 0.35388107
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Accuracy on the trainin

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Load the training dataset
path_train = "transformed_megaset4_sentences.csv"
train_data = pd.read_csv(path_train).sample(1000000)

path_test = "final_test2_sentences.csv"
test_data = pd.read_csv(path_test).sample(100000)

# Initialize the StandardScaler
scaler = StandardScaler()

def tostr(x):
    return "word_"+str(x)

x_features = [tostr(i) for i in range(1, 30)]

# Prepare the features and target for the training data
X_train = train_data[x_features]
X_train_scaled = scaler.fit_transform(X_train)
y_train = train_data['generated']

# Prepare the features and target for the testing data
X_test = test_data[x_features]
X_test_scaled = scaler.transform(X_test)
y_test = test_data['generated']

# Initialize KNeighborsClassifier
print("Starting training")
knn_clf = KNeighborsClassifier(n_neighbors=51)

# Train the classifier on the scaled training data
knn_clf.fit(X_train_scaled, y_train)

# Make predictions on the scaled testing data
y_pred_test = knn_clf.predict(X_test_scaled)
# Make predictions on the scaled training data for overfitting check
y_pred_train = knn_clf.predict(X_train_scaled)

# Evaluate the accuracy on the testing data
accuracy_test = accuracy_score(y_test, y_pred_test)
# Evaluate the accuracy on the training data
accuracy_train = accuracy_score(y_train, y_pred_train)

# Output the accuracies
print(f"Accuracy on the training set: {accuracy_train * 100:.2f}%")
print(f"Accuracy on the test set: {accuracy_test * 100:.2f}%")

# Calculate the confusion matrix for test data to get false positive and false negative rates
conf_matrix_test = confusion_matrix(y_test, y_pred_test)
tn, fp, fn, tp = conf_matrix_test.ravel()

# Calculate rates
false_positive_rate = fp / (fp + tn) if (fp + tn) > 0 else 0
false_negative_rate = fn / (fn + tp) if (fn + tp) > 0 else 0

# Print the rates
print(f"False Positive Rate on test data: {false_positive_rate * 100:.2f}%")
print(f"False Negative Rate on test data: {false_negative_rate * 100:.2f}%")
print(f"Average error rate on test data: {(false_positive_rate*100 + false_negative_rate*100) / 2 :.2f}%")

score = 1 / false_negative_rate + 1 / false_positive_rate
print(f"Score: {score:.2f}")

# Print the confusion matrix for the test data
print("Confusion Matrix for the test data:")
print(conf_matrix_test)


Starting training
Accuracy on the training set: 85.67%
Accuracy on the test set: 83.10%
False Positive Rate on test data: 7.82%
False Negative Rate on test data: 36.35%
Average error rate on test data: 22.09%
Score: 15.54
Confusion Matrix for the test data:
[[62835  5331]
 [11572 20262]]
